In [2]:
import numpy as np
import pandas as pd
import nltk
from time import time
from sqlalchemy import create_engine
import re
import spacy
import markovify
import warnings
warnings.filterwarnings("ignore")

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


symbolic link created for C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en <<===>> C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [7]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

data = pd.read_sql_query('SELECT * FROM twitter', con=engine)

engine.dispose()

In [8]:
data.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [10]:
data.drop(columns=['index', 'tweet_id'], inplace=True)

In [9]:
def text_cleaner(text):
    text = re.sub(r'--', ' ',text)
    text = re.sub("[\[].*?[\]]",'',text)
    text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b', ' ',text)
    text = ' '.join(text.split())
    return text

In [26]:
text_data = text_cleaner(' '.join(data.text))
positive_data = text_cleaner(' '.join(data[data['airline_sentiment'] == 'positive']['text']))
negative_data = text_cleaner(' '.join(data[data['airline_sentiment'] == 'negative']['text']))

In [27]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(text_data)
positive_doc = nlp(positive_data)
negative_doc = nlp(negative_data)

In [28]:
sents = ' '.join([sent.text for sent in doc.sents if len(sent.text) > 1])
positive_sents = ' '.join([sent.text for sent in positive_doc.sents if len(sent.text) > 1])
negative_sents = ' '.join([sent.text for sent in negative_doc.sents if len(sent.text) > 1])

In [21]:
class POSifiedText(markovify.Text):
    
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

### Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Are the generated sentences exhibit the same negative sentiment?

In [30]:
negative_generator = POSifiedText(negative_sents, state_size=3)

In [31]:
for i in range(7):
    print(negative_generator.make_sentence())
    print('--------------')

Your service SUCKS # usairwayssuck @USAirways I do n't get sarcasm , making computers more and more like real people .
--------------
@united As a Million Miler I 've been waiting @SouthwestAir can you follow me I will DM you my confirmation number when I get no benefits ? ! @VirginAmerica congrats , you just made it Tues , I ca n't watch my pre - paid DirectTV .
--------------
@USAirways do n't think you guys could not wait minutes .
--------------
Step  do n't suck so much @SouthwestAir I 'm upset because we were lied to .
--------------
@JetBlue @fllairport best way to do this online .
--------------
Ca n't find or anywhere @JetBlue Noooo ! ! Why is every single one getting delayed ?
--------------
Not usual gr8 southwest customer service I 've ever seen that .
--------------


### Do the previous task this time bu using only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [32]:
positive_generator = POSifiedText(positive_sents, state_size=3)

In [35]:
for i in range(7):
    print(positive_generator.make_sentence())
    print('--------------')

@USAirways - Janet & amp ; an upgrade .
--------------
None
--------------
@AmericanAir will award me , air miles ! ! ! AA1679 Another successful journey , thanks for your excellent customer service 2day , BUT I have a complaint .
--------------
The new screens are laptop - large & amp ; carrying his daughter .
--------------
None
--------------
😉 @JetBlue flight . proud to fly Jet Blue @JetBlue Start including PTO in your getaway packages and I 'm sure it 's just a rock ...
--------------
Thank you ! ! ! ! That 's super helpful .
--------------


### This time train your Markov chain model using all the tweets and generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [36]:
all_tweets_generator = POSifiedText(sents, state_size=3)

In [37]:
for i in range(7):
    print(all_tweets_generator.make_sentence())
    print('--------------')

@SouthwestAir Anyone answering phone this morning at our plane was malfunctioning .
--------------
@AmericanAir I took the exact same plane again .
--------------
# eternity # 5amMisery @JetBlue should I bother contacting them ?
--------------
Thanks for a great flight @SouthwestAir is not being delayed because flight attendant did n't come to work !
--------------
@USAirways you are supposed to be in adjacent seats , not across isle .
--------------
@united worst flight experience I 've ever used thanks for nothing .
--------------
Where my free miles at ? ! @united of course this morning I see a non - stop flight between Laguardia / dal into a stop in STL .
--------------


If you isolate the sentiment of the text, the generated responses tended to match the tone of the text fairly well. When trained on text that is more generalized, the responses kind of make sense in context, but the tone of the text can go either way (one sentence starts using language that would be considered having a positive sentiment, but the generated ending's tone doesn't match that even though the sentence structure seems otherwise normal)